In [ ]:
from sklearn.preprocessing import StandardScaler, RobustScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from category_encoders import CountEncoder, HashingEncoder
# define numeric features
numeric_features = df.select_dtypes(include=[np.number]).columns.tolist()
one_hot_features = ['Channel','CustomerOccupation','TransactionType']
freq_features = ['Location','MerchantID']
hash_features = ['AccountID','DeviceID','IP Address']
df.drop(columns=['TransactionID','TransactionDate','PreviousTransactionDate'])
# define transformers 
num_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

robust_features = ['AccountBalance', 'TransactionAmount_log']
robust_transformer = Pipeline(steps=[
    ('robust', RobustScaler())
])

onehot_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

hash_transformer = Pipeline(steps=[
    ('encoder',HashingEncoder(n_components=16))
])

freq_transformer = Pipeline(steps=[
    ('encoder',CountEncoder())
])

# combine into a column transformer
preprocessor = ColumnTransformer(transformers=[
    ('num',num_transformer, numeric_features),
    ('robust',robust_transformer,robust_features),
    ('onehot',onehot_transformer,one_hot_features),
    ('freq',freq_transformer, freq_features),
    ('hash',hash_transformer,hash_features)
],remainder='passthrough', verbose_feature_names_out=False)

In [ ]:
# main train function
